In [ ]:
# pip install pytorch-lightning==2.0.2
# pip install lightning-bolts==0.6.0.post1

In [9]:
# pip install git+https://github.com/PytorchLightning/lightning-bolts.git@master --upgrade

In [15]:
from pytorch_lightning import LightningModule, Trainer, seed_everything
# from torchmetrics import Accuracy
import torch.nn as nn
import os
import torch
import torch.nn.functional as F
from torch.optim import Adam
from torchmetrics.functional import accuracy
from torch.optim.lr_scheduler import OneCycleLR
import torchvision
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar

seed_everything(7)

PATH_DATASETS = os.environ.get("PATH_DATASETS", "./data/")
BATCH_SIZE = 256 if torch.cuda.is_available() else 64
NUM_WORKERS = int(os.cpu_count() / 2)


Global seed set to 7


In [16]:
class CustomRes(LightningModule):
    def __init__(self):
        super().__init__()
        self.prepLayer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3), stride=1, padding=1, bias=False),## in_feature_map = 32, out_feature_map = 32
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.Layer1_X1 = nn.Sequential(
            nn.Conv2d(in_channels =64, out_channels=128, kernel_size=(3,3), stride=1, padding=1, bias=False), #out_feature_map = 32
            nn.MaxPool2d(2), #  out_feature_map = 16
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        self.Layer1_R1 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride=1, padding=1, bias=False),#  out_feature_map = 16
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride=1, padding=1, bias=False),#  out_feature_map = 16
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        self.Layer2 = nn.Sequential(
            nn.Conv2d(in_channels =128, out_channels=256, kernel_size=(3,3), stride=1, padding=1, bias=False), ##  out_feature_map = 16
            nn.MaxPool2d(2),##  out_feature_map = 8
            nn.BatchNorm2d(256),
            nn.ReLU()
        ) 
        self.Layer3_X2 = nn.Sequential(
            nn.Conv2d(in_channels =256, out_channels=512, kernel_size=(3,3), stride=1, padding=1, bias=False), #out_feature_map = 8
            nn.MaxPool2d(2), #  out_feature_map = 4
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.Layer3_R2 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=1, padding=1, bias=False),#  out_feature_map = 4
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=1, padding=1, bias=False),#  out_feature_map = 4
            nn.BatchNorm2d(512),
            nn.ReLU()
        )   
        self.pool = nn.MaxPool2d(4) #  out_feature_map = 1
        self.fc = nn.Linear(512, 10)

    def forward(self,X):
        X = self.prepLayer(X)
        X = self.Layer1_X1(X)
        R1 = self.Layer1_R1(X)
        X = R1+X
        X = self.Layer2(X)
        X = self.Layer3_X2(X)
        R2 = self.Layer3_R2(X)
        X = R2+X
        X = self.pool(X)
        X = X.view(-1,512)
        X = self.fc(X)
        X = F.log_softmax(X,dim=-1)
        return X
    
    def training_step(self, batch, batch_idx):
        data, label = batch
        loss = F.nll_loss(self(data), label)
        return loss

    def evaluate(self, batch, stage=None):
        x,y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = logits.argmax(dim=1)
        acc = accuracy(preds, y)
        if stage:
            self.log(f'{stage}_loss',loss,prog_bar=True)
            self.log(f'{stage}_acc',acc, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, 'val')
    
    def configure_optimizers(self):
        optimizer = Adam(self.parameters, self.hparams.lr)
        steps_per_epoch=4500
        schedular = OneCycleLR(optimizer=optimizer, max_lr = .1, epochs=self.trainer.max_epochs, steps_per_epoch=steps_per_epoch)
        schedular_dict = {'scheduler':schedular,'interval':'step'}
        return {'optimizer':optimizer, 'lr_scheduler':schedular_dict}

In [17]:

train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ]
)

test_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),
    ]
)

cifar10_dm = CIFAR10DataModule(
    data_dir=PATH_DATASETS,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    val_transforms=test_transforms,
)

In [18]:

model = CustomRes()
trainer = Trainer(
    max_epochs=30,
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
    logger=CSVLogger(save_dir="logs/"),
    callbacks=[LearningRateMonitor(logging_interval="step"), TQDMProgressBar(refresh_rate=10)],
)

trainer.fit(model, cifar10_dm)
trainer.test(model, datamodule=cifar10_dm)

TypeError: `devices` selected with `CPUAccelerator` should be an int > 0.